## Database statisctics

In [14]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

%matplotlib inline

In [15]:
def create_bucket(df):
    print("Creating the buckets...")
    buckets = {}
    # Reading the buckets
    df_buckets = df[df['dup_id'] == '[]']
    loop = tqdm(total=df_buckets.shape[0])
    for row in df_buckets.iterrows():
        name = row[1]['bug_id']
        buckets[name] = set()
        buckets[name].add(name)
        loop.update(1)
    loop.close()
    # Fill the buckets
    df_duplicates = df[df['dup_id'] != '[]']
    loop = tqdm(total=df_duplicates.shape[0])
    for row_bug_id, row_dup_id in df_duplicates[['bug_id', 'dup_id']].values:
        bucket_name = int(row_dup_id)
        dup_id = row_bug_id
        while bucket_name not in buckets:
            query = df_duplicates[df_duplicates['bug_id'] == bucket_name]
            if query.shape[0] <= 0: 
                break
            bucket_name = int(query['dup_id'])
        '''
            Some bugs duplicates point to one master that
            does not exist in the dataset like openoffice master=152778
        '''
        if bucket_name in buckets:
            buckets[bucket_name].add(dup_id)
        loop.update(1)
    loop.close()
    return buckets

def read_pairs(file_path):
    n = 0
    with open(file_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        for row in f:
            pairs = row.split(' ')
            n += len(pairs) // 2
    return n

def getting_pairs(array):
    res = []
    for row in array:
        _, dups = row
        dups = list(dups)
        while len(dups) > 1:
            bucket = dups[0]
            dups.remove(bucket)
            for d in dups:
                res.append([bucket, d])
    return res

### Datasets

In [37]:
domains = ['eclipse', 'netbeans', 'openoffice']

### Eclipse

In [81]:
DATASET_ECLIPSE = os.path.join('data/normalized/{}'.format(domains[0]), '{}.csv'.format(domains[0]))
df_e = pd.read_csv(DATASET_ECLIPSE)

#### Bug status

In [45]:
df_e.groupby('bug_status').count()

,bug_id,bug_severity,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
bug_status,,,,,,,,,,,,
CLOSED,98800,98800,98800,98800,98800,98800,98800,98800,98800,98800,98800,98800
RESOLVED,219479,219479,219479,219479,219479,219478,219479,219479,219479,219479,219479,219479
VERIFIED,43605,43605,43605,43605,43605,43605,43605,43605,43605,43605,43605,43605


#### Bug Resolution

In [46]:
df_e.groupby('resolution').count()

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,short_desc,version
resolution,,,,,,,,,,,,
DUPLICATE,39545,39545,39545,39545,39545,39545,39545,39545,39545,39545,39545,39545
FIXED,234897,234897,234897,234897,234897,234897,234897,234897,234897,234897,234897,234897
INVALID,23102,23102,23102,23102,23102,23102,23101,23102,23102,23102,23102,23102
MOVED,21,21,21,21,21,21,21,21,21,21,21,21
NDUPLICATE,6320,6320,6320,6320,6320,6320,6320,6320,6320,6320,6320,6320
NOT_ECLIPSE,3530,3530,3530,3530,3530,3530,3530,3530,3530,3530,3530,3530
WONTFIX,29182,29182,29182,29182,29182,29182,29182,29182,29182,29182,29182,29182
WORKSFORME,25287,25287,25287,25287,25287,25287,25287,25287,25287,25287,25287,25287


#### Empty textual features

In [65]:
total_issues_no_description = df_e[ df_e['description'] == ' ' ].shape[0]
total_issues_no_title = df_e[ df_e['short_desc'] == ' ' ].shape[0]

print("Total issues no description: {} ({}%)".format(total_issues_no_description, round((total_issues_no_description / df_e.shape[0]) * 100.0), 2))
print("Total issues no title: {} ({}%)".format(total_issues_no_title, round((total_issues_no_title / df_e.shape[0]) * 100.0), 2))

Total issues no description: 3059 (1%)
Total issues no title: 0 (0%)


#### Export eclipse dataset

In [94]:
df_e_filtered = df_e[df_e['resolution'].isin(['DUPLICATE', 'FIXED', 'MOVED', 'NDUPLICATE', 'WONTFIX', 'WORKSFORME'])]
df_e_filtered.groupby('resolution').count()
df_e_filtered.to_csv(DATASET_ECLIPSE, index=False)

### Netbeans

In [83]:
DATASET_NETBEANS = os.path.join('data/normalized/{}'.format(domains[1]), '{}.csv'.format(domains[1]))
df_n = pd.read_csv(DATASET_NETBEANS)

#### Bug status

In [68]:
df_n.groupby('bug_status').count()

,bug_id,bug_severity,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
bug_status,,,,,,,,,,,,
CLOSED,37854,37854,37854,37854,37854,37753,37854,37854,37854,37854,37853,37854
RESOLVED,126603,126603,126603,126603,126603,126603,126603,126603,126603,126603,126603,126603
VERIFIED,52258,52258,52258,52258,52258,52258,52258,52258,52258,52258,52258,52258


#### Bug resolution

In [70]:
df_n.groupby('resolution').count()

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,short_desc,version
resolution,,,,,,,,,,,,
DUPLICATE,36232,36232,36232,36232,36232,36232,36232,36232,36232,36232,36232,36232
FIXED,120608,120608,120608,120608,120608,120608,120522,120608,120608,120608,120608,120608
INCOMPLETE,4839,4839,4839,4839,4839,4839,4839,4839,4839,4839,4839,4839
INVALID,15198,15198,15198,15198,15198,15198,15194,15198,15198,15198,15197,15198
NDUPLICATE,13,13,13,13,13,13,13,13,13,13,13,13
WONTFIX,19832,19832,19832,19832,19832,19832,19826,19832,19832,19832,19832,19832
WORKSFORME,19993,19993,19993,19993,19993,19993,19988,19993,19993,19993,19993,19993


In [72]:
df_n[df_n['resolution'] == 'INCOMPLETE']

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
1114,2501,normal,RESOLVED,applet,1999-07-15 20:59:00 +0000,2011-09-19 23:22:34 +0000,[jglick] There were some rumours of Sun coming...,[],P4,obsolete,INCOMPLETE,Suggest providing support in Applet module for...,3.x
7531,10209,blocker,RESOLVED,rmi,2001-03-13 09:34:00 +0000,2011-09-19 23:23:19 +0000,Description: \nit is usefull to have Startup a...,[],P4,obsolete,INCOMPLETE,RMID Wrapper could have Startup action in IDE,3.x
8707,11392,blocker,RESOLVED,vcsgeneric,2001-04-13 13:45:00 +0000,2011-09-19 23:23:27 +0000,NetBeans release 3.2 build #29 (RC2) on Window...,[],P4,obsolete,INCOMPLETE,"ALL: The meaning of ""Unix Shell"" settings sho...",3.x
9101,11794,blocker,RESOLVED,vcscore,2001-04-26 18:35:00 +0000,2011-09-19 23:23:34 +0000,"[NB32-38, RC4]\n\nthere has been an Issue #115...",[],P4,obsolete,INCOMPLETE,Better handling (de)serialization: contextual too,3.x
9841,12540,blocker,RESOLVED,rmi,2001-05-31 21:13:00 +0000,2011-09-19 23:00:02 +0000,This suggestion is based on my experience usin...,[],P3,obsolete,INCOMPLETE,Force generation of stubs for marked Java file,3.x
10027,12731,blocker,RESOLVED,vcsgeneric,2001-06-10 17:04:00 +0000,2011-09-19 23:00:23 +0000,Could the CVS Checkout Select be extended to m...,[],P3,obsolete,INCOMPLETE,"CVS Checkout Select, cater for PARTIAL & WILDC...",3.x
10916,13633,blocker,RESOLVED,netbrowser,2001-07-16 16:50:00 +0000,2011-09-19 22:42:51 +0000,"Currently, we use JTidy to get ""corrected"" pag...",[],P2,obsolete,INCOMPLETE,Loading page is synchronous,3.x
11231,13949,blocker,RESOLVED,vcscore,2001-07-26 10:06:00 +0000,2011-09-19 23:23:44 +0000,in general all command line based vcs tools sh...,[],P4,obsolete,INCOMPLETE,"Support ""Command-Line Style""",3.x
12707,15451,blocker,RESOLVED,Code,2001-09-13 15:30:00 +0000,2011-09-19 23:19:01 +0000,a FS should be converted before a JSP compilation,[],P3,javaee,INCOMPLETE,a FS should be converted before a JSP compilation,3.x
12899,15644,blocker,RESOLVED,corba,2001-09-19 15:27:00 +0000,2011-09-19 23:23:53 +0000,Now implementation generator generates impleme...,[],P4,obsolete,INCOMPLETE,Action for faster access to implementations,3.x


#### Empty textual features

In [71]:
total_issues_no_description = df_n[ df_n['description'] == ' ' ].shape[0]
total_issues_no_title = df_n[ df_n['short_desc'] == ' ' ].shape[0]

print("Total issues no description: {} ({}%)".format(total_issues_no_description, round((total_issues_no_description / df_n.shape[0]) * 100.0), 2))
print("Total issues no title: {} ({}%)".format(total_issues_no_title, round((total_issues_no_title / df_n.shape[0]) * 100.0), 2))

Total issues no description: 2 (0%)
Total issues no title: 0 (0%)


### OpenOffice

In [91]:
DATASET_OPENOFFICE = os.path.join('data/normalized/{}'.format(domains[2]), '{}.csv'.format(domains[2]))
df_o = pd.read_csv(DATASET_OPENOFFICE)

#### Bug status

In [86]:
df_o.groupby('bug_status').count()

,bug_id,bug_severity,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
bug_status,,,,,,,,,,,,
CLOSED,89999,89999,89999,89999,89999,89999,89999,89999,89999,89999,89999,89999
RESOLVED,5159,5159,5159,5159,5159,5159,5159,5159,5159,5159,5159,5159
VERIFIED,2912,2912,2912,2912,2912,2912,2912,2912,2912,2912,2912,2912


#### Bug resolution

In [87]:
df_o.groupby('resolution').count()

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,short_desc,version
resolution,,,,,,,,,,,,
DUPLICATE,14567,14567,14567,14567,14567,14567,14567,14567,14567,14567,14567,14567
FIXED,48033,48033,48033,48033,48033,48033,48033,48033,48033,48033,48033,48033
IRREPRODUCIBLE,13910,13910,13910,13910,13910,13910,13910,13910,13910,13910,13910,13910
MOVED,5,5,5,5,5,5,5,5,5,5,5,5
NDUPLICATE,5095,5095,5095,5095,5095,5095,5095,5095,5095,5095,5095,5095
NOT_AN_ISSUE,11926,11926,11926,11926,11926,11926,11926,11926,11926,11926,11926,11926
OBSOLETE,191,191,191,191,191,191,191,191,191,191,191,191
WONTFIX,4343,4343,4343,4343,4343,4343,4343,4343,4343,4343,4343,4343


In [88]:
df_o[df_o['resolution'].isin(['OBSOLETE'])]

,bug_id,bug_severity,bug_status,component,creation_ts,delta_ts,description,dup_id,priority,product,resolution,short_desc,version
3831,3988,trivial,CLOSED,ui,2002-04-12 02:38:00 +0000,2013-12-04 15:13:58 +0000,At present the only elements that are common t...,[],P3,Writer,OBSOLETE,relax constraint that objects and frames in he...,641
7741,8250,trivial,RESOLVED,code,2002-10-11 17:22:00 +0000,2013-08-15 16:43:54 +0000,Could anyone help.\n\nI have a macro written i...,[],P3,Writer,OBSOLETE,Importing a Word97 Macro,OOo 1.0.1
9078,9699,major,RESOLVED,ui,2002-11-30 18:00:00 +0000,2013-10-31 06:34:36 +0000,Openoffice 643C is installedin my windows 98 s...,[],P4,Writer,OBSOLETE,CRASH when insert rows into table with maximum...,643C
11502,12375,trivial,RESOLVED,code,2003-03-15 16:17:00 +0000,2013-09-19 09:38:54 +0000,OO 1.02 was working fine until I renamed my ho...,[],P3,Installation,OBSOLETE,Repair doesn't work,OOo 1.0.2
12115,13071,trivial,RESOLVED,ui,2003-04-05 20:29:00 +0000,2013-08-15 16:46:32 +0000,It would be nice to have audio feedback to spe...,[],P3,ui,OBSOLETE,Audio Feedback,OOo 1.1 Beta
14597,15904,trivial,CLOSED,nl,2003-06-22 07:48:00 +0000,2013-07-11 20:55:43 +0000,Als je bij het gebruik van de Nederlandstalige...,[],P3,Native-Lang,OBSOLETE,Fouten in de vertaling van het gebruikersinter...,OOo 1.1
15294,16628,trivial,CLOSED,nl,2003-07-09 22:17:00 +0000,2013-07-12 17:10:30 +0000,De GUI elementen die overeenkomen met de nieuw...,[],P3,Native-Lang,OBSOLETE,Onvolledige vertaling GUI 1.1 in het Nederlands,OOo 1.1 RC
18136,20064,trivial,CLOSED,code,2003-09-25 14:59:00 +0000,2013-07-27 10:42:16 +0000,Product Requirement:\nReduce retraining. Tools...,[],P4,General,OBSOLETE,Q-PCD Remove Main Toolbar,OOo 1.0.0
18174,19958,trivial,CLOSED,Website general issues,2003-09-23 23:52:00 +0000,2013-08-05 15:13:59 +0000,hi\nthe awards page needs to be updated and ne...,[],P3,Infrastructure,OBSOLETE,update awards page,current
19971,22240,minor,RESOLVED,ui,2003-11-07 11:56:00 +0000,2013-12-31 07:58:31 +0000,On PC the OOo-native Open and Save dialogs do ...,[],P3,ui,OBSOLETE,OOo's Open&Save Dialogs should show folder sho...,OOo 1.1


#### Empty textual features

In [89]:
total_issues_no_description = df_o[ df_o['description'] == ' ' ].shape[0]
total_issues_no_title = df_o[ df_o['short_desc'] == ' ' ].shape[0]

print("Total issues no description: {} ({}%)".format(total_issues_no_description, round((total_issues_no_description / df_o.shape[0]) * 100.0), 2))
print("Total issues no title: {} ({}%)".format(total_issues_no_title, round((total_issues_no_title / df_o.shape[0]) * 100.0), 2))

Total issues no description: 0 (0%)
Total issues no title: 0 (0%)


#### Export OpenOffice dataset

In [93]:
df_o_filtered = df_o[df_o['resolution'].isin(['DUPLICATE', 'FIXED', 'MOVED', 'NDUPLICATE', 'WONTFIX', 'OBSOLETE'])]
df_o_filtered.groupby('resolution').count()
df_o_filtered.to_csv(DATASET_OPENOFFICE, index=False)

### Total of bugs and pairs

In [38]:
df = pd.DataFrame(columns=['domain', 'train_pairs', 'test_pairs', 'total_bugs'])

rows = []

for DOMAIN in domains:
    METHOD = 'baseline'
    DIR = 'data/processed/{}'.format(DOMAIN)
    DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
    TRAIN = os.path.join(DIR, 'train.txt')
    TEST = os.path.join(DIR, 'test.txt')
    
    dataset = pd.read_csv(DATASET)
    
    n_train = read_pairs(TRAIN)
    n_test = read_pairs(TEST)
    
    rows.append({ 'domain' : DOMAIN, 'train_pairs' : n_train, 'test_pairs' : n_test, 'total_bugs' :  dataset.shape[0] })
    
df.append(rows)

,domain,train_pairs,test_pairs,total_bugs
0,eclipse,79073,7591,361006
1,netbeans,87543,7930,216715
2,openoffice,53740,4549,98070


### Creating split 90% train and 10% test

In [118]:
rows = []

for DOMAIN in domains:
    METHOD = 'baseline'
    DIR = 'data/processed/{}'.format(DOMAIN)
    DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
    
    dataset = pd.read_csv(DATASET)
    buckets = create_bucket(dataset)
    bucket_dups = []

    for key in buckets:
        if len(buckets[key]) > 1:
            bucket_dups.append([key, list(buckets[key])])
            
    pairs = getting_pairs(bucket_dups)
    
    VALIDATION_SPLIT = 0.9
    split_idx = int(len(pairs) * VALIDATION_SPLIT)

    with open(os.path.join(DIR, 'train_chronological.txt'), 'w') as f:
        for pair in pairs[:split_idx]:
            f.write("{} {}\n".format(pair[0], pair[1]))

    test_data = {}
    for pair in pairs[split_idx:]:
        bug1 = int(pair[0])
        bug2 = int(pair[1])
        if bug1 not in test_data:
            test_data[bug1] = set()
        test_data[bug1].add(bug2)
    with open(os.path.join(DIR, 'test_chronological.txt'), 'w') as f:
        for bug in test_data.keys():
            f.write("{} {}\n".format(bug, ' '.join([str(x) for x in test_data[bug]])))
    print('Train and test created')
    
    n_train = len(pairs[:split_idx])
    n_test = len(pairs[split_idx:])
    
    rows.append({ 'domain' : DOMAIN, 'train_pairs' : n_train, 'test_pairs' : n_test, 'total_bugs' :  dataset.shape[0] })
    
df = pd.DataFrame(data=rows, columns=['domain', 'train_pairs', 'test_pairs', 'total_bugs'])

Creating the buckets...



Train and test created
Creating the buckets...



Train and test created
Creating the buckets...



Train and test created


In [119]:
df

,domain,train_pairs,test_pairs,total_bugs
0,eclipse,78182,8687,361006
1,netbeans,85355,9484,216715
2,openoffice,52020,5781,98070


### Visualize a single split

In [26]:
DOMAIN = 'eclipse'
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
DIR = 'data/processed/{}'.format(DOMAIN)
dataset = pd.read_csv(DATASET)

In [27]:
buckets = create_bucket(dataset)

Creating the buckets...


In [28]:
len(buckets)

321536

In [29]:
bucket_dups = []

for key in buckets:
    if len(buckets[key]) > 1:
        bucket_dups.append([key, list(buckets[key])])

In [30]:
len(bucket_dups)

24414

In [31]:
pairs = getting_pairs(bucket_dups)

In [32]:
pairs[:10]

[[1, 183],
 [15392, 2],
 [15392, 9779],
 [15392, 94],
 [2, 9779],
 [2, 94],
 [9779, 94],
 [42962, 7],
 [10, 121067],
 [40, 20]]

In [33]:
len(pairs)

86863

In [34]:
VALIDATION_SPLIT = 0.9
split_idx = int(len(pairs) * VALIDATION_SPLIT)

with open(os.path.join(DIR, 'train_chronological.txt'), 'w') as f:
    for pair in pairs[:split_idx]:
        f.write("{} {}\n".format(pair[0], pair[1]))
        
test_data = {}
for pair in pairs[split_idx:]:
    bug1 = int(pair[0])
    bug2 = int(pair[1])
    if bug1 not in test_data:
        test_data[bug1] = set()
    test_data[bug1].add(bug2)
with open(os.path.join(DIR, 'test_chronological.txt'), 'w') as f:
    for bug in test_data.keys():
        f.write("{} {}\n".format(bug, ' '.join([str(x) for x in test_data[bug]])))
print('Train and test created')

Train and test created


In [35]:
print("Train: {}".format(len(pairs[:split_idx])))
print("Test: {}".format(len(pairs[split_idx:])))

Train: 78176
Test: 8687
